In [60]:
from imutils.video import FPS
from imutils.video import FileVideoStream
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import time
from imutils.video import count_frames 
SHAPE = [1080,1920]
vid_file='03.06.20 13DPH.mp4'#'11.12.2019 - frames 64935-65415.avi'#
output_vid_name='trial1.avi'

In [61]:
def get_centroid(x, y, w, h):
    """ Get bounding box centroid"""
    x1 = int(w / 2)
    y1 = int(h / 2)

    cx = x + x1
    cy = y + y1

    return (cx, cy)

def get_contours(image,min_width=50,min_height=50):
    """ Get the blobs or contours detected in the image
    input: 
    image - a frame
    min_width - the minimal width of a blob, all blobs below threshold will be disregarded
    min_height - the minimal width of a blob, all blobs below threshold will be disregarded
    returns: 
    matches - a list of tuples, each tuple contains two tuples with information about each detected blob.
        first tuple is the bounding box coordinates, width and height 
        second tuple contains the centroid coordinates
    """
    matches = []
    # find all contours/blobs in the frame:
    contours, hierarchy = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_TC89_L1)
    for (i, contour) in enumerate(contours):
        # for each detected object/blob/contour
        # Get the bounding box:
        (x, y, w, h) = cv2.boundingRect(contour)
        # Filter by height and width:
        contour_valid = (w >= min_width) and (
                    h >= min_height)

        if not contour_valid:
            continue
        # Get bbox centroid:
        centroid = get_centroid(x, y, w, h)
        matches.append(((x, y, w, h), centroid))
    return matches

def draw_boxes(img, pathes):
    """ Draw bounding boxes around objects in image
    """
    BOUNDING_BOX_COLOUR = (255, 10, 0) #BGR
    CENTROID_COLOUR = (255, 192, 0) #BGR
    for (i, match) in enumerate(pathes):
        contour, centroid = match
        x, y, w, h = contour

        cv2.rectangle(img, (x, y), (x + w - 1, y + h - 1),
                          BOUNDING_BOX_COLOUR, 4)
        cv2.circle(img, centroid, 2, CENTROID_COLOUR, -1)
    return img

def get_filter(frame,bg_sub,brighten=True,blur=False):
    """ Get the foreground mask for a frame
    input:
    frame - frame from video
    bg_sub - an open-cv pre-trained background subtractor
    brighten - bool, whether to adjust brightness in the image
    blur - bool, whether to apply gaussian blur to remove background noise from the image
    output:
    combined - a foreground mask created by fine-tuning the background subtractor output with edge detection.
    """
    gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #Turn to grayscale 
    denoise=cv2.GaussianBlur(gray,(71,71),0) 
    if blur:
        # Apply gaussian blur to image
        gray=denoise
    if brighten:
        # Apply brightness adjustment to image
        gray=cv2.convertScaleAbs(gray, alpha=1, beta=50)
    fg_mask = bg_sub.apply(gray, None, 0.001) # calculate foreground mask
    img=cv2.Canny(denoise,10,10) # get edges from the blurred image to filter out small floating particles
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10,10)) 
    closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel) # fill in gaps in the edges
    # get the areas that are detected by both the bg-sub and the edge detection routine:
    combined=cv2.bitwise_and(fg_mask,closing) 
    return combined

def train_bg_subtractor(cap, bg_sub, num, brighten=True, blur=False):
    """ Pre-train the background subtractor.
    input:
    cap - video capture object, the video
    bg_sub - background subtractor object
    num - number of frames to train on
    brighten - bool, whether to adjust brightness in the image
    blur - bool, whether to apply gaussian blur to remove background noise from the image
    output:
    bg_sub - trained background subtractor
    """
    for i in range(num):
        # iterate over the selected number of frames
        #ret,frame=cap.read() # get one frame
        _,frame=cap.read()
        gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # convert to grayscale 
        if brighten:
            # modify brightness if applicable:
            gray=cv2.convertScaleAbs(gray, alpha=1, beta=50)
        if blur:
            # apply gaussian blur if applicable:
            gray=cv2.GaussianBlur(gray,(71,71),0)
        fg_mask=bg_sub.apply(gray, None, 0.001) #apply bg_sub to the frame (modified or not)
    return bg_sub 

def save_frame(frame, file_name, flip=True):
    """ Save  frame as image"""
    # flip BGR to RGB
    if flip:
        cv2.imwrite(file_name, np.flip(frame, 2))
    else:
        cv2.imwrite(file_name, frame)
        
        
def process_vid(vid_name,output_vid,brighten=True,blur=False):
    """ Detect objects in a single video and create a new video with the bounding boxes around objects.
    input:
    vid_name - path and filename of the input video
    output_vid - path and filename for the generated video
    brighten - bool, whether to adjust brightness in the image
    blur - bool, whether to apply gaussian blur to remove background noise from the image
    """
    # count the number of frames in the video, so we can determine the portion dedicated for training
    # the background subtractor:
    num_frames=count_frames(vid_name) 
    cap=cv2.VideoCapture(vid_name) # open video
    train_frames=round(num_frames/4) # number of frames to pre-train the bg_sub on
    bg_sub=cv2.createBackgroundSubtractorMOG2(history=train_frames,detectShadows=True) 
    bg_sub=train_bg_subtractor(cap, bg_sub,train_frames) # pre-training
    images=[]
    # codec for video writing, see: https://www.pyimagesearch.com/2016/02/22/writing-to-video-with-opencv/
    fourcc = cv2.VideoWriter_fourcc(*"MJPG") 
    # the video writer is defined by output filename, selected codec, the fps selected is the same as the input video, 
    # frame size that is also identical to input video, and a boolean specific whether to save a color video:
    writer = cv2.VideoWriter(output_vid, fourcc, cap.get(cv2.CAP_PROP_FPS), (SHAPE[1], SHAPE[0]), True)
    # iterate over the remaining frames:
    while True:
        grabbed,frame=cap.read() # get frame
        if not grabbed:
            break
        mask=get_filter(frame,bg_sub) # get foreground mask
        matches=get_contours(mask) # find objects inside the mask, get a list of their bounding boxes
        img1=[] 
        img1=draw_boxes(frame,matches) # draw bounding boxes on original frame
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB) # optional: open-cv work in BGR so convert back to RGB
        writer.write(img1) # Write frame to the new video file
        #fps.update()
   # release resources:     
    writer.release()
    cap.release()


    

In [ ]:
fps = FPS().start()
process_vid(vid_file,output_vid_name,brighten=False,blur=True)
fps.stop()
print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

In [62]:
# cut out an example vid for Roi
c=cv2.VideoCapture(output_vid_name)
images=[]
fourcc = cv2.VideoWriter_fourcc(*"MJPG") 
writer = cv2.VideoWriter('fish_example.avi', fourcc, c.get(cv2.CAP_PROP_FPS), (SHAPE[1], SHAPE[0]), True)

for i in range(30):
    _,frame=c.read()
    #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    writer.write(frame)
    #images.append(Image.fromarray(frame))
writer.release()
c.release()
#images[0].save('fish.gif',
 #              save_all=True, append_images=images[1:], optimize=False, duration=40, loop=0)

In [59]:

print("[INFO] starting video file thread...")
num_frames=count_frames(vid_file) 
train_frames=round(num_frames/4) # number of frames to pre-train the bg_sub on
bg_sub=cv2.createBackgroundSubtractorMOG2(history=train_frames,detectShadows=True) 
bg_sub=train_bg_subtractor(fvs, bg_sub,train_frames)
fvs = FileVideoStream(vid_file).start()
time.sleep(1.0)
# start the FPS timer

fps = FPS().start()# loop over frames from the video file stream
while fvs.more():
    # grab the frame from the threaded video file stream, resize
    # it, and convert it to grayscale (while still retaining 3
    # channels)
    frame = fvs.read()
    mask=get_filter(frame,bg_sub) # get foreground mask
    matches=get_contours(mask) # find objects inside the mask, get a list of their bounding boxes
    img1=[] 
    img1=draw_boxes(frame,matches) # draw bounding boxes on original frame
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB) # optional: open-cv work in BGR so convert back to RGB
     #   writer.write(img1) # Write frame to the new video file
    #fps.update()
    # display the size of the queue on the frame
    #cv2.putText(frame, "Queue Size: {}".format(fvs.Q.qsize()),
     #   (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    # show the frame and update the FPS counter
    #cv2.imshow("Frame", frame)
    #cv2.waitKey(1)
    fps.update()
# stop the timer and display FPS information
fps.stop()
print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
# do a bit of cleanup
cv2.destroyAllWindows()
fvs.stop()

[INFO] starting video file thread...


error: OpenCV(4.3.0) /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


TypeError: new() missing 1 required positional argument: 'size'

In [63]:
count_frames('fish_example.avi')


30

In [25]:
frame

NameError: name 'frame' is not defined